In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.font_manager as fm

df = pd.read_csv("출자구조.CSV", encoding="cp949")

In [2]:
df

,피소유 기관,소유 기관,지분,비고
0,인천국제공항공사,국토교통부,100.00,NaN
1,한국가스공사,정부,26.15,NaN
2,한국가스공사,한국전력공사,20.47,NaN
3,한국가스공사,지방자치단체,7.39,NaN
4,한국가스공사,우리사주,1.07,NaN
...,...,...,...,...
106,기업은행,수출입은행,1.84,NaN
107,기업은행,기타,31.46,NaN
108,수출입은행,정부,76.38,NaN
109,수출입은행,산업은행,16.72,NaN


In [3]:
#한글 폰트 오류 방지
font_path = 'C:\\Windows\\Fonts\\malgun.ttf'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False 

In [4]:
UserInput = ['한국전력공사', '정부']

In [5]:
#함수 정의
def target_owners(df, target_orgs):
    results = {}
    for org in target_orgs:
        owners = df[df['피소유 기관'] == org]['소유 기관'].tolist()
        if owners:
            results[org] = owners
            sub_results = target_owners(df, owners)
            for key, value in sub_results.items():
                results[key] = value
    return results

def target_owning(df, target_orgs):
    results = {}
    for org in target_orgs:
        owned = df[df['소유 기관'] == org]['피소유 기관'].tolist()
        if owned:
            results[org] = owned
            sub_results = target_owning(df, owned)
            for key, value in sub_results.items():
                results[key] = value
    return results


In [7]:
targetOwners = target_owners(df, UserInput)
targetOwns = target_owning(df, UserInput)

print(f"각 기관의 지분을 가진 기관:\n{targetOwners}")
print(f"\n각 기관이 지분을 가진 기관:\n{targetOwns}")


각 기관의 지분을 가진 기관:
{'한국전력공사': ['정부', '산업은행', '국민연금공단', '외국인', '기타'], '산업은행': ['기획재정부', '국토교통부', '산업통상자원부', '해양수산부', '기후대응기금']}

각 기관이 지분을 가진 기관:
{'한국전력공사': ['한국가스공사', '한국남동발전', '한국남부발전', '한국동서발전', '한국서부발전', '한국수력원자력', '한국중부발전', '한국지역난방공사'], '정부': ['한국가스공사', '한국석유공사', '한국전력공사', '대한석탄공사', '한국광해광업공단', '한국수자원공사', '한국철도공사', '한국토지주택공사', '한국주택금융공사', '한국해양진흥공사', '기업은행', '수출입은행'], '한국수자원공사': ['한국해외인프라도시개발지원공사'], '한국철도공사': ['한국해외인프라도시개발지원공사'], '한국토지주택공사': ['한국해외인프라도시개발지원공사'], '한국해양진흥공사': ['부산항만공사', '여수광양항만공사', '울산항만공사', '인천항만공사'], '수출입은행': ['여수광양항만공사', '기업은행']}


In [9]:
DI = nx.DiGraph()

# 2. 모든 노드(기관) 추가
all_nodes = set(targetOwners.keys()) | set(targetOwns.keys())
for owners in targetOwners.values():
    all_nodes.update(owners)
for owned in targetOwns.values():
    all_nodes.update(owned)
DI.add_nodes_from(all_nodes)
DI.add_nodes_from(UserInput)
print(all_nodes)

{'한국남동발전', '한국서부발전', '기타', '여수광양항만공사', '산업통상자원부', '기획재정부', '한국전력공사', '한국수자원공사', '한국광해광업공단', '한국남부발전', '한국철도공사', '한국주택금융공사', '인천항만공사', '한국토지주택공사', '한국중부발전', '울산항만공사', '국토교통부', '한국동서발전', '해양수산부', '한국해외인프라도시개발지원공사', '한국석유공사', '한국지역난방공사', '산업은행', '국민연금공단', '한국해양진흥공사', '한국가스공사', '대한석탄공사', '한국수력원자력', '부산항만공사', '수출입은행', '외국인', '기후대응기금', '정부', '기업은행'}


In [ ]:
DI = nx.DiGraph()

# 2. adding nodes
all_nodes = set(targetOwners.keys()) | set(targetOwns.keys())
for owners in targetOwners.values():
    all_nodes.update(owners)
for owned in targetOwns.values():
    all_nodes.update(owned)
DI.add_nodes_from(all_nodes)
DI.add_nodes_from(UserInput)

# 3. edges classification
edges_userOwner = []
edges_userOwns = []

for owner, owned_list in targetOwners.items():
    for owned in owned_list:
        DI.add_weighted_edges_from(owner, owned)
        if owner in UserInput:
            edges_userOwner.append((owner, owned))
        else:
            edges_userOwns.append((owner, owned))

for owned, owner_list in all_owners_dict.items():
    for owner in owner_list:
        G.add_edge(owner, owned)
        if owner in UserInput:
            user_owned_edges.append((owner, owned))
        else:
            user_owner_edges.append((owner, owned))
